In [ ]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    tqdm \
    pinecone-notebooks==0.1.1 \
    python-dotenv

In [10]:
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()
from pinecone_connection import PineConeDB

# Access the API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
EMBED_DIM = 3072

# Create index 
# Initialize Pinecone

pc = PineConeDB(PINECONE_API_KEY)
hasaki_index = pc.create_index(index_name='hasaki-index-v2', dimension=EMBED_DIM)


## Upsert campaign-index (Updated)

In [2]:
campaigns = pd.read_csv('../../../data/results/campaigns.csv')
campaigns.rename(columns={'Unnamed: 0': 'campaignID'}, inplace=True)
campaigns.head()

,campaignID,label,link,img
0,0,"Khai trương chi nhánh 221, 222 - Cần Thơ, Đồng...",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240-172...
1,1,"Khai trương chi nhánh 223, 224 - Hội An, TP.HCM",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240mobi...
2,2,Sale lương về - Deal ngập tràn,https://hasaki.vn/campaign/best-seller,https://media.hcdn.vn/hsk/campaign/640x240---2...
3,3,[HOT] Ưu đãi trải nghiệm làm đẹp đến hơn 60%,https://hasaki.vn/campaign/trai-nghiem-dich-vu,https://media.hcdn.vn/hsk/campaign/wap16981146...
4,4,Unilever Nâng Niu Nét Đẹp Toàn Diện,https://hasaki.vn/campaign/unilever-khuyen-mai...,https://media.hcdn.vn/hsk/campaign/UnileverSIS...


In [3]:
from pipecone_connection import create_vector_emb

campaigns['vector'] = campaigns['label'].apply(lambda x: create_vector_emb(x))
campaigns.head()

/home/phatpham/anaconda3/envs/rag-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,campaignID,label,link,img,vector
0,0,"Khai trương chi nhánh 221, 222 - Cần Thơ, Đồng...",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240-172...,"[0.09607130289077759, 0.31030184030532837, -0...."
1,1,"Khai trương chi nhánh 223, 224 - Hội An, TP.HCM",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240mobi...,"[-0.19367045164108276, -0.004278192762285471, ..."
2,2,Sale lương về - Deal ngập tràn,https://hasaki.vn/campaign/best-seller,https://media.hcdn.vn/hsk/campaign/640x240---2...,"[0.031419891864061356, 0.08881331980228424, -0..."
3,3,[HOT] Ưu đãi trải nghiệm làm đẹp đến hơn 60%,https://hasaki.vn/campaign/trai-nghiem-dich-vu,https://media.hcdn.vn/hsk/campaign/wap16981146...,"[0.16664496064186096, 0.32210487127304077, -0...."
4,4,Unilever Nâng Niu Nét Đẹp Toàn Diện,https://hasaki.vn/campaign/unilever-khuyen-mai...,https://media.hcdn.vn/hsk/campaign/UnileverSIS...,"[-0.10489341616630554, 0.008368105627596378, -..."


In [6]:
# Prepare and upsert data
campaigns_list = [{"id": str(row['campaignID']), 
         "values": row['vector'],
         "metadata": {'label': row['label'], 'link': row['link'], 'img': row['img']}} for _, row in campaigns.iterrows()]
hasaki_index.upsert(vectors=campaigns_list,
                    namespace='campaign-namespace')


{'upserted_count': 13}

In [ ]:
# ref: https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/gen-qa-openai.ipynb#scrollTo=KPXw1sEubwoF

## Upsert support-indexn (Updated)

In [7]:
support = pd.read_csv('../../../data/results/supports.csv')
# support = support.drop(columns=['Unnamed: 0'], axis=1)
support.rename(columns={'Unnamed: 0': 'supportID'}, inplace=True)

support.head()

,supportID,title,link,content
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...


In [9]:
from pipecone_connection import create_vector_emb

support['vector'] = support['content'].apply(lambda x: create_vector_emb(x))
support.head()

,supportID,title,link,content,vector
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,"[0.01456688717007637, 0.33099138736724854, -0...."
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...,"[0.07909272611141205, 0.3365171551704407, -0.3..."
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...,"[0.07966456562280655, 0.3297780156135559, -0.4..."
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...,"[0.035692255944013596, 0.3778902292251587, -0...."
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...,"[0.02337840385735035, 0.2968924045562744, -0.3..."


In [14]:
import re
# .apply(lambda x: re.sub(r'<[^>]+>', '', x))
# Prepare and upsert data
support_list = [{"id": str(row['supportID']), 
         "values": row['vector'],
         "metadata": {'title': re.sub(r'<[^>]+>', '', row['title']), 'link': row['link'], 'content': row['content']}} for _, row in support.iterrows()]

hasaki_index.upsert(vectors=support_list,
                    namespace = 'support-namespace')

{'upserted_count': 27}

## Upsert category-index (Updated)

In [16]:
categories = pd.read_csv('../../../data/results/categories.csv')
categories.rename(columns={'Unnamed: 0': 'categoryID'}, inplace=True)
categories.head()

,categoryID,link,img,cat
0,0,https://hasaki.vn/danh-muc/trang-diem-moi-c24....,https://media.hcdn.vn/catalog/category/c24-tra...,Trang Điểm Môi
1,1,https://hasaki.vn/danh-muc/mat-na-c30.html,https://media.hcdn.vn/catalog/category/30_1_im...,Mặt Nạ
2,2,https://hasaki.vn/danh-muc/trang-diem-mat-c52....,https://media.hcdn.vn/catalog/category/c52-tra...,Trang Điểm Mặt
3,3,https://hasaki.vn/danh-muc/sua-rua-mat-c19.html,https://media.hcdn.vn/catalog/category/19_3_im...,Sữa Rửa Mặt
4,4,https://hasaki.vn/danh-muc/trang-diem-vung-mat...,https://media.hcdn.vn/catalog/category/c50-tra...,Trang Điểm Mắt


In [17]:
from pipecone_connection import create_vector_emb
categories['vector'] = categories['cat'].apply(lambda x: create_vector_emb(x))
categories.head()

,categoryID,link,img,cat,vector
0,0,https://hasaki.vn/danh-muc/trang-diem-moi-c24....,https://media.hcdn.vn/catalog/category/c24-tra...,Trang Điểm Môi,"[0.04586426913738251, -0.06818058341741562, 0...."
1,1,https://hasaki.vn/danh-muc/mat-na-c30.html,https://media.hcdn.vn/catalog/category/30_1_im...,Mặt Nạ,"[-0.05931249260902405, 0.08366265147924423, -0..."
2,2,https://hasaki.vn/danh-muc/trang-diem-mat-c52....,https://media.hcdn.vn/catalog/category/c52-tra...,Trang Điểm Mặt,"[-0.016898442059755325, -0.3402135968208313, -..."
3,3,https://hasaki.vn/danh-muc/sua-rua-mat-c19.html,https://media.hcdn.vn/catalog/category/19_3_im...,Sữa Rửa Mặt,"[-0.11072025448083878, -0.45872682332992554, -..."
4,4,https://hasaki.vn/danh-muc/trang-diem-vung-mat...,https://media.hcdn.vn/catalog/category/c50-tra...,Trang Điểm Mắt,"[-0.3483051359653473, -0.42001697421073914, -0..."


In [18]:
# Prepare and upsert data
category_list = [{"id": str(row['categoryID']), 
         "values": row['vector'],
         "metadata": {'link': row['link'], 'img': row['img'], 'cat': row['cat']}} for _, row in categories.iterrows()]
hasaki_index.upsert(vectors=category_list,
              namespace="category-namespace")


{'upserted_count': 15}

## Upsert product-pname-index

In [2]:
import pandas as pd
product_data = pd.read_csv('../../../data/results/product_data_v2.csv')
product_data.drop(columns={'Unnamed: 0'}, inplace=True)
product_data.shape

(2868, 20)

In [86]:
product_data = product_data.drop(columns=['price_extended', 'about', 'combo'])
product_data.columns

Index(['pid', 'plink', 'pname', 'pcat', 'price', 'delivery', 'meta',
       'ingredients', 'usage', 'img', 'desc', 'count_NEG', 'count_NEU',
       'count_POS', 'cmt_summary_NEG', 'cmt_summary_NEU', 'cmt_summary_POS'],
      dtype='object')

In [87]:
product_data = product_data.drop_duplicates(subset=['pid'])
product_data.shape

(2811, 17)

In [88]:
product_data['price'] = product_data['price'].apply(lambda x: int(re.sub(r'\D', '', x)))
product_data['delivery'] = product_data['delivery'].apply(lambda x: x.replace('\n', ' '))

In [ ]:
product_data.head()

In [1]:
import pandas as pd
product_data = pd.read_csv('../../../data/results/preprocessed_product_211124.csv')
product_data

,plink,pid,pname,pcat,price,variant,brand,description,rating
0,https://hasaki.vn/dich-vu/trai-nghiem-combo-la...,105375,Trải Nghiệm Combo Lấy Mụn Vùng Mặt + Chiếu ASS...,Dịch Vụ Phòng Khám / Điều Trị Mụn,149000,1 lần | 55 phút,Unknown,Trải Nghiệm Combo Lấy Mụn Vùng Mặt + Chiếu ASS...,4.8
1,https://hasaki.vn/dich-vu/dieu-tri-mun-bang-ip...,122214,Điều Trị Mụn Bằng IPL (Cơ Bản),Dịch Vụ Phòng Khám / Điều Trị Mụn,500000,1 lần | 40 phút,Unknown,"Mụn lên ồ ạt, da tiết dầu nhờn không kiểm soát...",5.0
2,https://hasaki.vn/dich-vu/trai-nghiem-peel-bha...,107984,"Điều Trị Mụn, Giảm Thâm Bằng BHA 20%",Dịch Vụ Phòng Khám / Điều Trị Mụn,500000,1 lần | 43 phút,Unknown,"Tối ưu hiệu quả trị mụn viêm, mụn trứng cá,......",4.8
3,https://hasaki.vn/dich-vu/xoa-not-ruoi-cong-ng...,33051,Xóa Nốt Ruồi Công Nghệ Laser Fractional CO2,Dịch Vụ Phòng Khám / Điều Trị Mọi Bệnh Lý Về Da,100000,1 Lần | 5 phút,Unknown,Xóa nốt ruồivới công nghệ Laser Fractional C02...,4.3
4,https://hasaki.vn/dich-vu/trai-nghiem-dieu-tri...,99719,Trải Nghiệm Điều Trị Sẹo Lõm Phương Pháp 5in1,Dịch Vụ Phòng Khám / Điều Trị Sẹo Rỗ,2200000,1 lần | 90 phút,Unknown,Điều Trị Sẹo Lõm Phương Pháp 5in1 - Tăng Tỉ Lệ...,5.0
...,...,...,...,...,...,...,...,...,...
5875,https://hasaki.vn/san-pham/son-kem-li-gilaa-ri...,97921,Son Kem Lì Gilaa 08 Take A Sip - Đỏ Rượu Vang 5g,Trang Điểm / Trang Điểm Môi / Son Kem / Tint,209000,08 Take A Sip,Gilaa,Son Kem Lì Gilaa Long Wear Lip Cream Rich Rosi...,5.0
5876,https://hasaki.vn/san-pham/son-kem-li-gilaa-th...,92035,Son Kem Lì Gilaa Thế Hệ Mới 02 Lovely Day - Đỏ...,Trang Điểm / Trang Điểm Môi / Son Kem / Tint,198000,02 Lovely Day,Gilaa,Son Kem Lì Gilaa Long Wear Lip Cream 5glà dòng...,4.7
5877,https://hasaki.vn/san-pham/hsd-09-2025-son-kem...,135820,[HSD 09/2025] Son Kem Lì Gilaa Thế Hệ Mới 03 S...,Clearance Sale,136000,5g,Gilaa,Xem thêm nội dung,0.0
5878,https://hasaki.vn/san-pham/son-moi-gilaa-duong...,94537,Son Môi Gilaa Dưỡng Căng Mọng Môi #00 Lip Plum...,Trang Điểm / Trang Điểm Môi / Son Kem / Tint,275000,00 Lip Plumper,Gilaa,Son Môi Gilaa Plumping Lip Serum 3.6glà dòngso...,5.0


In [2]:
product_data['description'] = product_data['description'].fillna('Unknown')

In [3]:
product_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5880 entries, 0 to 5879
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   plink        5880 non-null   object 
 1   pid          5880 non-null   int64  
 2   pname        5880 non-null   object 
 3   pcat         5645 non-null   object 
 4   price        5880 non-null   int64  
 5   variant      5744 non-null   object 
 6   brand        5880 non-null   object 
 7   description  5880 non-null   object 
 8   rating       5876 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 413.6+ KB


In [4]:
from pc_utils import create_vector_emb
product_data['vector'] = product_data['pname'].apply(lambda x: create_vector_emb(x))
product_data['desc_vector'] = product_data['description'].apply(lambda x: create_vector_emb(x))

/home/phatpham/anaconda3/envs/rag-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
product_data.head()

,plink,pid,pname,pcat,price,variant,brand,description,rating,vector,desc_vector
0,https://hasaki.vn/dich-vu/trai-nghiem-combo-la...,105375,Trải Nghiệm Combo Lấy Mụn Vùng Mặt + Chiếu ASS...,Dịch Vụ Phòng Khám / Điều Trị Mụn,149000,1 lần | 55 phút,Unknown,Trải Nghiệm Combo Lấy Mụn Vùng Mặt + Chiếu ASS...,4.8,"[-0.03989139, -0.003975524, 0.007358804, -0.02...","[-0.014507835, 0.0034000622, -0.00081445713, -..."
1,https://hasaki.vn/dich-vu/dieu-tri-mun-bang-ip...,122214,Điều Trị Mụn Bằng IPL (Cơ Bản),Dịch Vụ Phòng Khám / Điều Trị Mụn,500000,1 lần | 40 phút,Unknown,"Mụn lên ồ ạt, da tiết dầu nhờn không kiểm soát...",5.0,"[-0.04654774, -0.037849806, 0.007563695, -0.00...","[-0.043247852, -0.0024392423, 0.0055921567, -0..."
2,https://hasaki.vn/dich-vu/trai-nghiem-peel-bha...,107984,"Điều Trị Mụn, Giảm Thâm Bằng BHA 20%",Dịch Vụ Phòng Khám / Điều Trị Mụn,500000,1 lần | 43 phút,Unknown,"Tối ưu hiệu quả trị mụn viêm, mụn trứng cá,......",4.8,"[-0.019837698, -0.023611384, 0.0061387015, 0.0...","[-0.024928644, -0.01626179, 0.011000238, -0.00..."
3,https://hasaki.vn/dich-vu/xoa-not-ruoi-cong-ng...,33051,Xóa Nốt Ruồi Công Nghệ Laser Fractional CO2,Dịch Vụ Phòng Khám / Điều Trị Mọi Bệnh Lý Về Da,100000,1 Lần | 5 phút,Unknown,Xóa nốt ruồivới công nghệ Laser Fractional C02...,4.3,"[-0.023302512, -0.066282704, 0.018656006, -0.0...","[-0.01815836, -0.036091458, 0.0075461683, -0.0..."
4,https://hasaki.vn/dich-vu/trai-nghiem-dieu-tri...,99719,Trải Nghiệm Điều Trị Sẹo Lõm Phương Pháp 5in1,Dịch Vụ Phòng Khám / Điều Trị Sẹo Rỗ,2200000,1 lần | 90 phút,Unknown,Điều Trị Sẹo Lõm Phương Pháp 5in1 - Tăng Tỉ Lệ...,5.0,"[-0.054176375, -0.027380617, 0.0099964645, -0....","[-0.0038773313, -0.033518378, -0.008143771, -0..."


In [6]:
product_data.to_csv('vectorized_product_211124.csv', index=False)

In [11]:
# Prepare and upsert data
product_list = [{"id": str(row['pid']), 
         "values": row['vector'],
         "metadata": {'product_id': row['pid'], 'pname': row['pname']}} for _, row in product_data.iterrows()]

# Define a batch size 
batch_size = 100  

for i in range(0, len(product_list), batch_size):
    batch = product_list[i:i + batch_size]
    hasaki_index.upsert(vectors=batch,
                        namespace='product-pname-namespace')

## Upsert product-desc-index (Updated)

In [12]:
# Prepare and upsert data
product_list = [{"id": str(row['pid']), 
         "values": row['desc_vector'],
         "metadata": {'product_id': row['pid'], 'pname': row['pname']}} for _, row in product_data.iterrows()]

batch_size = 100  

for i in range(0, len(product_list), batch_size):
    batch = product_list[i:i + batch_size]
    hasaki_index.upsert(vectors=batch,
                        namespace='product-desc-namespace')